In [ ]:
%matplotlib inline

[Pytorch Word Embeddings Tutorial](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)

In [4]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

ModuleNotFoundError: No module named 'torch'

In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


An Example: N-Gram Language Modeling
---

Recall that in an n-gram language model, given a sequence of words
$w$, we want to compute

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

Where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training
examples and update the parameters with backpropagation.


In [4]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
[520.9422106742859, 518.2166900634766, 515.5093951225281, 512.8182926177979, 510.1440649032593, 507.48634600639343, 504.842120885849, 502.2115545272827, 499.5938756465912, 496.989200592041]


Exercise: Computing Word Embeddings: Continuous Bag-of-Words
---

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.

Implement this model in Pytorch by filling in the class below. Some
tips:

* Think about which parameters you need to define.
* Make sure you know what shape each operation expects. Use .view() if you need to
  reshape.




### References

* https://iksinc.online/tag/continuous-bag-of-words-cbow/
* http://mccormickml.com/assets/word2vec/Alex_Minnaar_Word2Vec_Tutorial_Part_II_The_Continuous_Bag-of-Words_Model.pdf
* https://stackoverflow.com/questions/48479915/what-is-the-preferred-ratio-between-the-vocabulary-size-and-embedding-dimension
* https://github.com/FraLotito/pytorch-continuous-bag-of-words/blob/master/cbow.py
* https://stackoverflow.com/questions/50792316/what-does-1-mean-in-pytorch-view
* https://www.tensorflow.org/tutorials/text/word_embeddings
* https://pytorch.org/docs/stable/nn.html

In [97]:
# CBOW implementation

# Size of the context windows, 2 and 5 are supposed to be used in ex02...
# range \in [2, 1/2 * document_length - 1]
CONTEXT_SIZE = 2

# let's stick with this notation for now ;)
WINDOW_SIZE = CONTEXT_SIZE * 2

# Hyperparameter, we should determine this via sklearn.gridsearch equiv in pytorch
# nvm, the ex02.pdf mentions it should be 50
EMBEDDING_DIM = 50

# document
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# "Vocabulary"
vocab = set(raw_text)    
vocab_size = len(vocab)

# Vocabulary (mapping of words to ids)
word_to_ix = {word: i for i, word in enumerate(vocab)}  

# Data creation - get context around the target word
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    # Context before w_i
    context_before_w = raw_text[i - CONTEXT_SIZE: i]
    
    # Context after w_i
    context_after_w = raw_text[i + 1: i + CONTEXT_SIZE + 1]
    
    # Put them together
    context_window = context_before_w + context_after_w
    
    # Target = w_i
    target = raw_text[i]
    
    # Append in the correct format
    data.append((context_window, target))


class CBOW(nn.Module):

    
    def __init__(self, vocab_size, embedding_dim, context_size, nr_hidden_neurons=128):
        super(CBOW, self).__init__()
        # nr_hidden_neurons can be determined via gridsearch
        
        # Embedding layer
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # note: this probably doesn't deal with 'UNK' words
        self.linear1 = nn.Linear(embedding_dim, nr_hidden_neurons)  
        
        # output layer
        self.linear2 = nn.Linear(nr_hidden_neurons, vocab_size)

        
    def forward(self, inputs):
        # shape = (WINDOW_SIZE, EMBEDDING_DIM)
        embeds = self.embeddings(inputs)
    
        # sum over all of the context vectors
        # shape = (EMBEDDING_DIM)
        summed_embeds = sum(embeds)

        # shape = (1, EMBEDDING_DIM)
        # -1 param in view() ... "the actual value for this dimension will be inferred so that the number of elements in the view matches the original number of elements."
        embeds_2D = summed_embeds.view(1, -1)
        
        # finally compute the hidden layer weighted sum (a.k.a. output before using the activation function)
        # ... and don't forget to divide by the number of input vectors
        h =  self.linear1(embeds_2D) / WINDOW_SIZE
        
        # output of the hidden layer
        out =  F.relu(h) 
         
        
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

# create your model and train.  here are some functions to help you make
# the data ready for use by your module
# creates a tensor with ids of the context words
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


example_context_vector_ids = make_context_vector(data[0][0], word_to_ix)   # example
print(data[0][0])
print(example_context_vector_ids)
print(vocab)

# <word in vocab>: <id in word_to_ix>
# 'We': 19, 'are': 35, 'to': 4, ...


losses = []
loss_function = nn.NLLLoss() # negative log likelihood loss
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)

optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(100):
    total_loss = 0
    for context, target in data:

        # Step1. Create input vector (vector of ids of context words)
        context_vector_ids = make_context_vector(context, word_to_ix)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_vector_ids)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
    

print(losses)  
# 0.1 learning_rate doin' some work on this small document!

['We', 'are', 'to', 'study']
tensor([19, 35,  4, 34])
{'As', 'called', 'program.', 'we', 'to', 'is', 'spells.', 'process.', 'the', 'The', 'processes.', 'our', 'spirits', 'a', 'computational', 'create', 'idea', 'directed', 'effect,', 'We', 'In', 'other', 'of', 'direct', 'process', 'evolution', 'things', 'Computational', 'computers.', 'programs', 'by', 'about', 'data.', 'conjure', 'study', 'are', 'processes', 'that', 'they', 'inhabit', 'beings', 'pattern', 'evolve,', 'rules', 'manipulate', 'computer', 'with', 'abstract', 'People'}
[229.72310781478882, 197.22341990470886, 161.6775110065937, 120.19694504141808, 77.4002638310194, 41.28086268901825, 19.01519876718521, 9.087207017466426, 5.181709220632911, 3.6868550768122077, 2.895898678340018, 2.38931486569345, 2.033222970087081, 1.769405565224588, 1.5645083147101104, 1.4011681880801916, 1.2683989179786295, 1.1579408384859562, 1.064459912944585, 0.984705597627908, 0.9154239704366773, 0.8551724227145314, 0.8020715599413961, 0.7546046967618167

In [17]:
import spacy

In [30]:
#EXERCISE For WORD EMBEDDINGS
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

nlp = spacy.load('en')

def spacy_tokenize(text, tokenizer):
    tokens = list(tokenizer(text))
    return [str(i) for i in tokens]

text = "Mary, don’t slap the green witch"
print(list(spacy_tokenize(text, nlp)))

      
raw_texts = spacy_tokenize(raw_text, nlp)
print(type(raw_texts))
    

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_texts)
print(vocab)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
print(word_to_ix)
data = []
for i in range(2, len(raw_texts) - 2):
    context = [raw_texts[i - 2], raw_texts[i - 1],
               raw_texts[i + 1], raw_texts[i + 2]]
    target = raw_texts[i]
    data.append((context, target))
print(data[:5])


class CBOW(nn.Module):

    def __init__(self, vocab_size=0, context_size=2, embedding_size=15):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.linear1 = nn.Linear(embedding_size, vocab_size)

    def forward(self, inputs):
        lookup_embeds = self.embeddings(inputs)
        embeds = lookup_embeds.sum(dim=0)
        out = self.linear1(embeds)
        out = F.log_softmax(out)
        return out


# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example

model = CBOW()

['Mary', ',', 'do', 'n’t', 'slap', 'the', 'green', 'witch']
<class 'list'>
{'idea', 'pattern', 'manipulate', 'directed', 'other', 'abstract', 'is', 'effect', 'rules', 'beings', 'direct', ',', 'a', 'the', 'evolve', 'People', 'are', 'our', 'to', 'called', 'spirits', 'we', 'about', 'process', 'create', 'In', 'with', 'they', 'The', 'computers', '\n', 'data', 'things', 'inhabit', 'Computational', 'program', 'study', 'conjure', 'As', 'of', '.', 'evolution', 'computational', 'We', 'programs', 'that', 'computer', 'processes', 'by', 'spells'}
{'idea': 0, 'pattern': 1, 'manipulate': 2, 'directed': 3, 'other': 4, 'abstract': 5, 'is': 6, 'effect': 7, 'rules': 8, 'beings': 9, 'direct': 10, ',': 11, 'a': 12, 'the': 13, 'evolve': 14, 'People': 15, 'are': 16, 'our': 17, 'to': 18, 'called': 19, 'spirits': 20, 'we': 21, 'about': 22, 'process': 23, 'create': 24, 'In': 25, 'with': 26, 'they': 27, 'The': 28, 'computers': 29, '\n': 30, 'data': 31, 'things': 32, 'inhabit': 33, 'Computational': 34, 'program':